In [5]:
import pandas as pd, numpy as np, sqlite3 as sql, datetime as dt, re, time, yfinance as yf
from dateutil.relativedelta import relativedelta
import nltk
import os, gc
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [6]:
from api_key import cryptor

In [1174]:
with sql.connect('../data/interim/companies.db') as con:
        port = pd.read_sql(f"SELECT Date date, Open, High, Low, Close, Volume, Volatility, Turnover, symbol FROM daily ORDER BY date", con=con, parse_dates={'date': '%Y-%m-%d %H:%M:%S'})\
                .drop_duplicates(subset=['date', 'symbol'])
        recommends = pd.read_sql(f"SELECT Date date, symbol, Firm, new_grade, prev_grade, Action from recommendations ORDER BY Date", con=con, parse_dates={'date': '%Y-%m-%d %H:%M:%S'})
        arts =pd.read_sql("SELECT date, symbol, publisher, pos_sent, neu_sent, neg_sent, comp_sent FROM articles ORDER BY date", con=con, parse_dates={'date': '%Y-%m-%d %H:%M:%S'})
        crypt_arts = pd.read_sql("SELECT date, symbol, publisher,pos_sent, neu_sent, neg_sent, comp_sent  FROM news_sentiment ORDER BY date", con=con, parse_dates={'date': '%Y-%m-%d %H:%M:%S'})
        articles = pd.concat([arts, crypt_arts], axis=0, ignore_index=True)
        comments = pd.read_sql(f"SELECT DATE(timestamp) date, channel, symbols, pos_sent, neu_sent, neg_sent, comp_sent from symbol_comments ORDER BY timestamp", parse_dates={'date': '%Y-%m-%d'}, con=con)
        comments.loc[:, "symbols"] = comments.symbols.apply(lambda x: x.replace('BTC', 'BTC-USD'))
        companies = tuple(port.symbol.unique())
        c_data = pd.read_sql(f"SELECT * from mentions WHERE symbol IN {companies}", con=con, index_col='pk')

In [1175]:
symbols_re = re.compile(r"\[|\]|\'|\'")
last_index = comments.index.max()

In [1165]:
# decompose for single symbol
for i, row in comments.iterrows():
    symbols = re.sub(symbols_re, "", row.symbols)
    symbols = symbols.split(',')
    for sym in symbols:
        last_index+=1
        comments.loc[last_index, ["symbols"]] = sym
        comments.loc[last_index, ["comment_index"]] = i
        comments.loc[last_index, ["date", "channel", "pos_sent", "neu_sent", "neg_sent", "comp_sent"]] = row.date, row.channel, row.pos_sent, row.neu_sent,  row.neg_sent, row.comp_sent


In [1195]:
print(time.perf_counter())

time.perf_counter()

75420.9830585


75421.0021197

In [ ]:
comments.assign(sym = lambda x: x.symbols.apply(lambda x: re.sub(symbols_re, '', x)).apply(str.split, sep=',')).explode('sym').reset_index

In [1166]:
comments = comments[lambda x:~( x.comment_index.isnull())]
comments = comments[lambda x: x.symbols.isin(companies)]

In [1167]:
recommendsDict = {"Very Bearish": 1, "Bearish": 2, "Neutral": 3, "Bullish": 4, "Very Bullish": 5}

In [1168]:
recommends=recommends.assign(new_sent = lambda x: x.new_grade.apply(lambda g: recommendsDict[g]))\
    .assign(prev_sent = lambda x: x.prev_grade.apply(lambda g: recommendsDict[g]))

In [1169]:
port.columns

Index(['date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Volatility',
       'Turnover', 'symbol'],
      dtype='object')

In [ ]:
def create_daily_data(ticker):
    tick = yf.ticker.Ticker(ticker)
    historical_data = tick.history("1mo")
    outstanding = tick.info.get("sharesOutstanding")
    if outstanding == None:
        outstanding = 1
    daily_close = historical_data["Close"]
    pct_change = daily_close.pct_change().fillna(0)
    periods = 2
    # calc volatility
    vola = (pct_change.rolling(periods).std() * np.sqrt(periods)).fillna(0)
    historical_data = historical_data.assign(Volatility = vola)
    historical_data = historical_data.assign(Turnover = lambda x: x.Volume / outstanding)
    historical_data = historical_data.assign(symbol = ticker)
    return historical_data.reset_index().round({"Volatility": 6, "Turnover": 6})

In [1170]:
# take aggregations over wanted frequency; make buy decisions based off of the frequency of data points and sentiments
# return port with new information: shares and cost * shares
class EAT():
    def __init__(self, portfolio, articles, comments, recs, start, end):
        self.portfolio = portfolio.copy(deep=True)
        self.postions = []
        self.start = start
        self.end = end
        self.articles = articles[lambda x: (x.date >= start) & (x.date <= end)]
        self.comments =  comments[lambda x: (x.date >= start) & (x.date <= end)]
        self.recs = recs[lambda x: (x.date >= start) & (x.date <= end)]

        self.aggs = {}

    def aggregate(self):
        articles_agg = self.articles.groupby([pd.Grouper(key="date", freq="1Y"), 'symbol'])\
            .agg({'pos_sent': ['mean'], 'neg_sent': ['mean'], 'neu_sent': ['mean'], 'comp_sent': ['mean', 'count']}).assign(type=lambda x: 'News')
        comments_agg = self.comments.groupby([pd.Grouper(key="date", freq="1Y"), 'symbols'])\
            .agg({'pos_sent': ['mean'], 'neg_sent': ['mean'], 'neu_sent': ['mean'], 'comp_sent': ['mean', 'count']}).assign(type=lambda x: 'Chats')
        recommends_agg = self.recs.groupby([pd.Grouper(key="date", freq="1Y"), 'symbol'])\
            .agg({'new_sent': ['mean'], 'prev_sent': ['mean', 'count']}).assign(type=lambda x: 'Analysts')
        recommends_agg = recommends_agg.reset_index()
        comments_agg = comments_agg.reset_index()
        articles_agg = articles_agg.reset_index()
        recommends_agg.columns = recommends_agg.columns.droplevel(1)
        comments_agg.columns = comments_agg.columns.droplevel(1)
        articles_agg.columns = articles_agg.columns.droplevel(1)
        
        articles_agg.columns = ['date', 'symbol', 'pos_sent', 'neg_sent', 'neu_sent', 'comp_sent',
       'counts', 'type']
        comments_agg.columns = ['date', 'symbol', 'pos_sent', 'neg_sent', 'neu_sent', 'comp_sent',
       'counts', 'type']
        recommends_agg.columns = ['date', 'symbol', 'new_sent', 'prev_sent', 'counts', 'type']
        # comments_agg=comments_agg.assign(date = lambda x: x.date.apply(lambda x: x.date))
        self.aggs['recommendations'] = recommends_agg
        self.aggs['articles'] = articles_agg
        self.aggs['comments'] = comments_agg
        return None 


    def tradeSents(self, agg, label, min_samples, min_comp_sent, shares):
        # add action, shares, cost
        returns = self.aggs[agg][lambda x: (x.date >= self.start) & (x[label] >= min_comp_sent) & (x.counts >= min_samples)]
        # query portfolio for first cost add columns
        indexes = pd.Int64Index([])
        for date, sym in returns.loc[:, ['date', 'symbol']].values:
            # ns = returns[lambda x: x.date == date].shape[0]
            if sym not in self.postions:
                self.postions.append(sym)
                f1_date = (date + relativedelta(years=1)).to_pydatetime()
                indexes = self.portfolio[lambda x: ((x.date > date) & (x.symbol == sym) & (x.date <= f1_date))].index
                self.portfolio.loc[indexes, "shares"] = shares
            else:
                self.postions.append(sym)
                f1_date = (date + relativedelta(years=1)).to_pydatetime()
                indexes = self.portfolio[lambda x: ((x.date > date) & (x.symbol == sym) & (x.date <= f1_date))].index
                self.portfolio.loc[indexes, "shares"] = shares * self.postions.count(sym)
            
            i = returns[lambda x: (x.date == date) & (x.symbol == sym)].index
            if not indexes.empty:
                returns.loc[i, 'cost'] = shares * self.portfolio.loc[indexes[0], "Close"]
                returns.loc[i, 'returns'] = shares * self.portfolio.loc[indexes[-1], "Close"]
            else:
                indexes = self.portfolio[lambda x: (x.symbol == sym)].index
                returns.loc[i, 'cost'] = shares * self.portfolio.loc[indexes[-1], "Open"]
                returns.loc[i, 'returns'] = shares * self.portfolio.loc[indexes[-1], "Close"]

        return self.portfolio#returns
        


In [1171]:
eat = EAT(port, articles, comments, recommends, dt.datetime(2018, 1, 1), dt.datetime(2022, 1, 30))

In [1172]:
eat.aggregate()

In [1173]:
eat.tradeSents("comments", "comp_sent", min_samples=1, min_comp_sent=0.15, shares=10)

C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,date,Open,High,Low,Close,Volume,Volatility,Turnover,symbol,shares
0,2017-01-03,42.972000,44.066002,42.192001,43.397999,29616500,0.000000,2.950000e-02,TSLA,NaN
1,2017-01-03,11.420000,11.650000,11.020000,11.430000,55182000,0.000000,4.570000e-02,AMD,NaN
2,2017-01-03,27.250112,27.374832,27.005378,27.332474,115127600,0.000000,7.000000e-03,AAPL,NaN
3,2017-01-03,20.639999,21.840000,20.532000,21.360001,73033,0.000000,4.000000e-04,ACB,NaN
4,2017-01-03,89.000000,89.000000,88.080002,88.599998,8789400,0.000000,3.200000e-03,BABA,NaN
...,...,...,...,...,...,...,...,...,...,...
98728,2022-01-28,24.003000,25.709999,22.809999,25.639999,21496600,0.101186,7.133600e-02,PTON,NaN
98729,2022-01-28,20.660000,21.320000,19.309999,20.900000,96497500,0.058454,6.066900e-02,NIO,NaN
98730,2022-01-28,1.850000,1.870000,1.750000,1.780000,2441000,0.035781,2.085600e-02,SOLO,NaN
98731,2022-01-29,37780.714844,38576.261719,37406.472656,38138.179688,17194183075,0.008032,1.719418e+10,BTC-USD,NaN


In [956]:
ret = eat.tradeSents("articles", "comp_sent", 100, 0.5, 10)

C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [957]:
ret = eat.tradeSents("recommendations", "new_sent", 25, 4, 10)

In [958]:
# eat.tradeSents("articles", "comp_sent", 100, 0.5, 10)
# eat.tradeSents("recommendations", "new_sent", 25, 4, 10)

In [1152]:
ret.groupby('date').sum().assign(r_pct = lambda x: (x.returns - x.cost) / x.cost)

,pos_sent,neg_sent,neu_sent,comp_sent,counts,cost,returns,r_pct
date,,,,,,,,
2019-12-31,5.265895,0.623934,15.110028,8.133532,93,42329.746435,67754.449959,0.600634
2020-12-31,4.164832,0.921737,24.733701,5.878339,1405,330494.633604,502545.053391,0.520585


In [1153]:
ret = eat.tradeSents("articles", "comp_sent", 100, 0.5, 10)
ret.groupby('date').sum().assign(r_pct = lambda x: (x.returns - x.cost) / x.cost)

C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,pos_sent,neg_sent,neu_sent,comp_sent,counts,cost,returns,r_pct
date,,,,,,,,
2020-12-31,1.886672,0.725528,19.387871,12.806051,6245,363631.928234,537816.954250,0.479015
2021-12-31,2.635620,0.961372,26.403193,18.722140,9305,90168.539557,73207.400317,-0.188105


In [1157]:
ret = eat.tradeSents("recommendations", "new_sent", 25, 4, 10)
ret.groupby('date').sum().assign(r_pct = lambda x: (x.returns - x.cost) / x.cost)

C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,new_sent,prev_sent,counts,cost,returns,r_pct
date,,,,,,
2018-12-31,29.418501,29.140315,280,21844.340649,27577.331295,0.262447
2019-12-31,33.888142,33.519803,355,30153.087006,47727.473297,0.582839
2020-12-31,67.845811,67.276012,993,57397.397575,62125.701008,0.082378
2021-12-31,58.988827,58.416372,694,67120.639572,56591.000690,-0.156876


In [808]:
# with sql.connect('../data/interim/companies.db') as con:
#     new_port.to_sql('daily', con=con, index=True, index_label="pk", if_exists='replace')

In [47]:
with sql.connect('../data/interim/companies.db') as con:
        port =pd.read_sql("SELECT * FROM daily ORDER BY date", con=con, parse_dates={'date': '%Y-%m-%d %H:%M:%S'})
        arts =pd.read_sql("SELECT * FROM articles ORDER BY date", con=con, parse_dates={'date': '%Y-%m-%d %H:%M:%S'})
        crypt_arts = pd.read_sql("SELECT * FROM crypt_articles ORDER BY date", con=sql.connect('../data/raw/crypt.db'))
        #articles = pd.concat([arts, crypt_arts], axis=0, ignore_index=True)
        comments = pd.read_sql(f"SELECT DATE(timestamp) date, channel, symbols, pos_sent, neu_sent, neg_sent, comp_sent from symbol_comments ORDER BY timestamp", parse_dates={'date': '%Y-%m-%d'}, con=con)
        comments.loc[:, "symbols"] = comments.symbols.apply(lambda x: x.replace('BTC', 'BTC-USD'))
        companies = tuple(port.symbol.unique())
        c_data = pd.read_sql(f"SELECT * from mentions WHERE symbol IN {companies}", con=con, index_col='pk')

In [48]:
decrypt_cols = [x for x in crypt_arts.columns if x not in ['pk', 'pos_sent', "neu_sent", "neg_sent", "comp_sent"]]
for col in decrypt_cols:
    crypt_arts.loc[:, col] = crypt_arts.loc[:, col].apply(bytes).apply(cryptor.decrypt).apply(str, encoding='utf-8')
    if col == 'date':
        crypt_arts.loc[:, col] = crypt_arts.loc[:, col].apply(str.split, sep=" ").apply(lambda x: x[0]).apply(pd.to_datetime)

In [49]:
crypt_arts

,pk,article,headline,comments,date,link,symbol,publisher,pos_sent,neg_sent,neu_sent,comp_sent
0,9,The Transportation Security Administration say...,TSA extends COVID mask rule for U.S. transport...,49,2021-08-17,https://seekingalpha.com/news/3731674-tsa-exte...,AAL,Seeking Alpha,0.042,0.042,0.916,0.2544
1,5705,"Parler, the social network popular with conser...",Parler comes back online after a month off the...,249,2021-02-16,https://seekingalpha.com/news/3662032-parler-c...,FB,Seeking Alpha,0.082,0.039,0.879,0.9363
2,1709,Amazon (NASDAQ:AMZN): Q1 GAAP EPS of $15.79 be...,"Amazon EPS beats by $6.18, beats on revenue",115,2021-04-29,https://seekingalpha.com/news/3688132-amazon-e...,AMZN,Seeking Alpha,0.065,0.000,0.935,0.7184
3,3240,"Blink Charging (BLNK +7.4%), EVgo (EVGO +8.4%)...",Electric vehicle battery stocks break higher a...,28,2021-07-28,https://seekingalpha.com/news/3720809-electric...,BLNK,Seeking Alpha,0.047,0.038,0.916,0.1280
4,5968,Teenagers across the U.S. are looking to Redmo...,TikTok rescue by Microsoft could make sense - ...,93,2020-08-01,https://seekingalpha.com/news/3598676-tiktok-r...,FB,Seeking Alpha,0.114,0.033,0.853,0.8934
...,...,...,...,...,...,...,...,...,...,...,...,...
15321,11885,"The Associated Press, NBC, Edison Research, Fo...",Joe Biden wins U.S. presidential election,23654,2020-11-07,https://seekingalpha.com/news/3633355-joe-bide...,QQQ,Seeking Alpha,0.059,0.048,0.893,0.0258
15322,9323,Animal spirits are firing up again on red-hot ...,Electric vehicle names head higher led by Kand...,126,2020-11-23,https://seekingalpha.com/news/3638678-electric...,NIO,Seeking Alpha,0.033,0.046,0.921,-0.2263
15323,2573,Boeing (BA +3.1%) bounces sharply higher after...,Boeing sales outpaced cancellations last month...,10,2021-03-09,https://seekingalpha.com/news/3670770-boeing-s...,BA,Seeking Alpha,0.021,0.066,0.913,-0.7005
15324,7455,"Workers in Arizona, Oregon, and New Mexico hav...",Intel workers file coronavirus safety complaints,15,2020-05-08,https://seekingalpha.com/news/3572138-intel-wo...,INTC,Seeking Alpha,0.133,0.087,0.779,0.9112


In [50]:
arts = arts.assign(comments = lambda x: 0)
arts = arts.loc[:, ['pk', 'article', 'title', 'comments','date', 'link', 'symbol', 'publisher','pos_sent', 'neg_sent', 'neu_sent', 'comp_sent']]

In [51]:
# nongreedy
regex = re.compile(r"© Reuters.+?(-|—)")

In [52]:
arts = arts.assign(article = lambda x: x.article.apply(lambda w: re.sub(regex, "", w)).apply(str.strip))

In [53]:
arts = arts.rename(columns={'title': 'headline'})

In [54]:
arted = pd.concat([crypt_arts.assign(sa=lambda x: 1), arts.assign(sa=lambda x: 0)], axis=0, ignore_index=True)

In [55]:
stop_words = set(nltk.corpus.stopwords.words("english"))
stop_words = stop_words.union({'said', 'us', 'also', 'inc', 'could', 'word', 'b', 'q', })

In [56]:
def wordCounts(df, comment_array):
    comment_list = []
    for comment in comment_array:
        comment_dictionary = {} 
        punc_regex = r"[\，\’\'\'\"\"\“\”!\?@#$%&\(\)\*,-.\\\{\}+~\/:;<>\[\]^`|=_]"
        reg_bad_quotes = re.compile(u"[\x94\x93\x92\x91]")
        comment = comment.lower()
        comment = re.sub(punc_regex, " ", comment)
        comment = re.sub(reg_bad_quotes, " ", comment)
        words = nltk.tokenize.word_tokenize(comment)
        for w in words:
            if (w in comment_dictionary.keys()) & (w not in stop_words):
                comment_dictionary[w] += 1
            elif (w not in stop_words):
                comment_dictionary[w] = 1
        comments_words = dict(sorted(comment_dictionary.items(), key=lambda x: x[1], reverse=True))
        comment_list.append(comments_words)
    
    df = df.assign(word_obj = pd.Series(comment_list))
    return df

# word counts sans stopwords

In [57]:
wordy_df = wordCounts(arted, arted.article.values)

In [58]:
wordy_df2 = wordy_df.copy(deep=True)

In [59]:
wordy_df2 = wordy_df2.assign(article = lambda x: x.article.apply(lambda line: re.sub(r'(?<=[.,])(?=[^\s])', r' ', line)))

In [60]:
wordy_df3 = wordy_df2.assign(word_obj = lambda x: x.word_obj.apply(str))

In [61]:
#wordy_df3.to_sql('temp_table', con=sql.connect('temp.db'), if_exists='replace')

In [62]:
wordy_df3

,pk,article,headline,comments,date,link,symbol,publisher,pos_sent,neg_sent,neu_sent,comp_sent,sa,word_obj
0,9,The Transportation Security Administration say...,TSA extends COVID mask rule for U.S. transport...,49,2021-08-17,https://seekingalpha.com/news/3731674-tsa-exte...,AAL,Seeking Alpha,0.042,0.042,0.916,0.2544,1,"{'transportation': 2, 'administration': 2, 'sa..."
1,5705,"Parler, the social network popular with conser...",Parler comes back online after a month off the...,249,2021-02-16,https://seekingalpha.com/news/3662032-parler-c...,FB,Seeking Alpha,0.082,0.039,0.879,0.9363,1,"{'parler': 7, 'users': 4, 'platform': 3, 'skys..."
2,1709,Amazon (NASDAQ:AMZN): Q1 GAAP EPS of $15. 79 b...,"Amazon EPS beats by $6.18, beats on revenue",115,2021-04-29,https://seekingalpha.com/news/3688132-amazon-e...,AMZN,Seeking Alpha,0.065,0.000,0.935,0.7184,1,"{'billion': 6, '5': 4, 'revenue': 3, '108': 3,..."
3,3240,"Blink Charging (BLNK +7. 4%), EVgo (EVGO +8. 4...",Electric vehicle battery stocks break higher a...,28,2021-07-28,https://seekingalpha.com/news/3720809-electric...,BLNK,Seeking Alpha,0.047,0.038,0.916,0.1280,1,"{'infrastructure': 5, 'bill': 5, '7': 4, 'ev':..."
4,5968,Teenagers across the U. S. are looking to Redm...,TikTok rescue by Microsoft could make sense - ...,93,2020-08-01,https://seekingalpha.com/news/3598676-tiktok-r...,FB,Seeking Alpha,0.114,0.033,0.853,0.8934,1,"{'would': 5, 'tiktok': 4, 'microsoft': 4, 'mak..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26805,7584,Stock in gaming and e-commerce firm Sea (NYSE:...,Sea Slumps as Tencent Moves to Cut Voting Stak...,0,2022-01-04,https://www.investing.com/news/stock-market-ne...,SE,Investing.com,0.084,0.046,0.870,0.7867,0,"{'tencent': 5, 'sea': 4, 'voting': 4, 'stake':..."
26806,1625,"On Wednesday, Palantir Technologies Inc (NYSE:...",Palantir Technologies Announces Collaboration ...,0,2022-01-05,https://www.investing.com/news/stock-market-ne...,PLTR,Investing.com,0.143,0.015,0.842,0.9630,0,"{'palantir': 6, 'platform': 5, 'hhi': 4, 'big'..."
26807,7583,Asia Pacific stocks were down on Thursday morn...,"Asian Stocks Down, Positive Chinese Data Fails...",0,2022-01-05,https://www.investing.com/news/stock-market-ne...,SE,Investing.com,0.068,0.034,0.898,0.9201,0,"{'services': 4, '1': 4, 'day': 4, 'u': 3, '0':..."
26808,9547,"The S&P 500 closed down Friday, marking its wo...",S&P 500 in Big Weekly Loss as Tech Bulls Scatt...,0,2022-01-07,https://www.investing.com/news/stock-market-ne...,ON,Investing.com,0.096,0.049,0.856,0.9733,0,"{'stocks': 5, 'nasdaq': 5, 'tech': 4, 'rate': ..."


In [63]:
bad_encode_regex = re.compile(u"[\x94\x93\x92\x91]")
wordy_df2 = wordy_df2.assign(article = lambda x: x.article.apply(lambda line: re.sub(r'(?<=[.,])(?=[^\s])', r' ', line)))
wordy_df2 = wordy_df2.assign(article = lambda x: x.article.apply(lambda line: re.sub(bad_encode_regex, r' ', line)))

In [64]:
def wordCounts(comment_array):
    comment_dictionary = {} 
    for comment in comment_array:
        punc_regex = r"[\，\’\'\'\"\"\“\”!\?@#$%&\(\)\*,-.\\\{\}+~\/:;<>\[\]^`|=_]"
        reg_bad_quotes = re.compile(u"[\x94\x93\x92\x91]")
        comment = comment.lower()
        comment = re.sub(punc_regex, " ", comment)
        comment = re.sub(reg_bad_quotes, " ", comment)
        comment = re.sub(r"[0-9]", "", comment)
        words = nltk.tokenize.word_tokenize(comment)
        for w in words:
            if (w in comment_dictionary.keys()) & (w not in stop_words):
                comment_dictionary[w] += 1
            elif (w not in stop_words):
                comment_dictionary[w] = 1
    
    comments_words = dict(sorted(comment_dictionary.items(), key=lambda x: x[1], reverse=True))
    # comment_list.append(comments_words)

    # df = df.assign(word_obj = pd.Series(comment_list))
    return pd.Series(comments_words)

# word counts sans stopwords

In [65]:
positive_article_words = wordCounts(wordy_df2[lambda x: x.comp_sent >= 0].article.values)

In [66]:
negative_article_words = wordCounts(wordy_df2[lambda x: x.comp_sent < 0].article.values)

In [67]:
negative_article_words = negative_article_words.reset_index().rename(columns={'index': 'word', 0: 'neg_counts'})

In [68]:
positive_article_words = positive_article_words.reset_index().rename(columns={'index': 'word', 0: 'pos_counts'})

In [69]:
positive_article_words[lambda x: x.word=='draftkings']

,word,pos_counts
739,draftkings,1114


In [70]:
negative_article_words[lambda x: x.word == 'draftkings']

,word,neg_counts
2779,draftkings,56


In [71]:
positive_example = wordCounts(wordy_df2[lambda x: (x.symbol == 'DKNG') & (x.comp_sent >= 0)].article).reset_index().rename(columns={'index': 'word', 0: 'neg_counts'})

In [72]:
wordy_df2[lambda x: (x.date <= dt.datetime(2021, 2, 1)) & (x.date >= dt.datetime(2021, 1, 1))].groupby('symbol').count().sort_values('date').loc['TSLA', :]

pk           115
article      115
headline     115
comments     115
date         115
link         115
publisher    115
pos_sent     115
neg_sent     115
neu_sent     115
comp_sent    115
sa           115
word_obj     115
Name: TSLA, dtype: int64

In [73]:
positive_article_words.merge(how='outer', right=negative_article_words).fillna(0).\
    assign(pos_pct = lambda x: x.pos_counts / (x.pos_counts + x.neg_counts)).\
        assign(neg_pct = lambda x: x.neg_counts / (x.pos_counts + x.neg_counts)).sort_values('neg_pct').\
            assign(counts = lambda x: (x.pos_counts + x.neg_counts)).sort_values('counts', ascending=False)


,word,pos_counts,neg_counts,pos_pct,neg_pct,counts
0,nasdaq,57260.0,12043.0,0.826227,0.173773,69303.0
1,nyse,41399.0,8421.0,0.830971,0.169029,49820.0
4,u,20841.0,6841.0,0.752872,0.247128,27682.0
2,company,21977.0,3720.0,0.855236,0.144764,25697.0
3,year,21347.0,3856.0,0.847002,0.152998,25203.0
...,...,...,...,...,...,...
51564,synbiotic,1.0,0.0,1.000000,0.000000,1.0
51563,minnett,1.0,0.0,1.000000,0.000000,1.0
51562,ord,1.0,0.0,1.000000,0.000000,1.0
51561,wfm,1.0,0.0,1.000000,0.000000,1.0


In [74]:
sia = SentimentIntensityAnalyzer()

In [75]:
def sentiment_art(art):
    return sia.polarity_scores(art)

In [76]:
wordy_df_news = wordy_df2.assign(sentiment_dict = lambda x : x.article.apply(sentiment_art))

In [77]:
wordy_df_news= wordy_df_news.assign(pos_sent = lambda x: x.sentiment_dict.apply(lambda y: y['pos']))\
    .assign(neg_sent = lambda x: x.sentiment_dict.apply(lambda y: y['neg']))\
        .assign(neu_sent = lambda x: x.sentiment_dict.apply(lambda y: y['neu']))\
            .assign(comp_sent = lambda x: x.sentiment_dict.apply(lambda y: y['compound']))

In [78]:
wordy_df_news.drop('sentiment_dict', axis=1, inplace=True)

In [79]:
wordy_df_news.groupby(pd.Grouper(key='date', freq="1M")).count()

,pk,article,headline,comments,link,symbol,publisher,pos_sent,neg_sent,neu_sent,comp_sent,sa,word_obj
date,,,,,,,,,,,,,
2018-01-31,1,1,1,1,1,1,1,1,1,1,1,1,1
2018-02-28,0,0,0,0,0,0,0,0,0,0,0,0,0
2018-03-31,0,0,0,0,0,0,0,0,0,0,0,0,0
2018-04-30,0,0,0,0,0,0,0,0,0,0,0,0,0
2018-05-31,0,0,0,0,0,0,0,0,0,0,0,0,0
2018-06-30,1,1,1,1,1,1,1,1,1,1,1,1,1
2018-07-31,0,0,0,0,0,0,0,0,0,0,0,0,0
2018-08-31,1,1,1,1,1,1,1,1,1,1,1,1,1
2018-09-30,0,0,0,0,0,0,0,0,0,0,0,0,0


In [88]:
comments#.groupby(pd.Grouper(key='date', freq='1M')).count()

,date,channel,symbols,pos_sent,neu_sent,neg_sent,comp_sent
0,2019-08-08,wetlqd-ideas,['AMD'],0.293,0.707,0.000,0.4404
1,2019-08-13,wetlqd-ideas,['MSFT'],0.182,0.818,0.000,0.4404
2,2019-08-13,wetlqd-ideas,['MSFT'],0.294,0.706,0.000,0.3612
3,2019-08-14,wetlqd-ideas,"['ON', 'MA', 'ING']",0.000,0.597,0.403,-0.4019
4,2019-08-14,wetlqd-ideas,['CRM'],0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...
3389,2020-12-08,option-trading,['OOOO'],0.000,1.000,0.000,0.0000
3390,2020-12-08,option-trading,['ING'],0.900,0.100,0.000,0.8402
3391,2020-12-08,trading,['FSLY'],0.000,1.000,0.000,0.0000
3392,2020-12-08,trading,['IT'],0.000,1.000,0.000,0.0000


In [622]:
with sql.connect('../data/interim/discord/discord.db') as con:
    full_comments =pd.read_sql(f"SELECT DATE(timestamp) date, channel, server, pk, content, isBot, mentions, emojis, links, chat_emotes from comments ORDER BY timestamp", parse_dates={'date': '%Y-%m-%d'}, con=con)
with sql.connect('../data/interim/companies.db') as con:
    wanted_companies = pd.read_sql("SELECT DISTINCT symbol FROM daily WHERE symbol NOT IN ('PT', 'IT', 'ON', 'ING')", con=con).symbol.values
    wanted_companies = tuple(wanted_companies)
    mentions = pd.read_sql(f"SELECT * from mentions WHERE symbol IN {wanted_companies}", con=con, index_col='pk')

In [623]:
full_comments

,date,channel,server,pk,content,isBot,mentions,emojis,links,chat_emotes
0,2019-08-02,wetlqd-ideas,Misc,0,Play account today,0,,,,
1,2019-08-02,wetlqd-ideas,Misc,1,@everyone today’s total. This doesn’t calculat...,0,everyone,,,
2,2019-08-02,wetlqd-ideas,Misc,2,Crack,0,,,,
3,2019-08-02,wetlqd-ideas,Misc,3,Big crack,0,,,,
4,2019-08-02,wetlqd-ideas,Misc,4,Heroin,0,,,,
...,...,...,...,...,...,...,...,...,...,...
31929,2020-12-08,trading,LSV,31929,<@!644997307976122388> all of my shorter video...,0,644997307976122388,,,
31930,2020-12-08,trading,LSV,31930,<@428921853403463713> have you done a video in...,0,428921853403463713,,,
31931,2020-12-08,trading,LSV,31931,"<@!184420431242067977> nope, not yet",0,184420431242067977,,,
31932,2020-12-08,trading,LSV,31932,they're the airline wifi company right?,0,,,,


In [624]:
userIdCatcher = r"(<@(!)?\d+>|@everyone)"
discord_emote_regex = r"<:.+:\d+>"
url_regex = r"https*\:.+|www\..+"
punc_regex = r"[\’\'\'\"\"\“\”!\?@#$%&\(\)\*,-.\\\{\}+~\/:;<>\[\]^`|=_’]"
contract_regex = r"[\'\’\’]"

In [625]:
def clean_comment(comment):
    comment = re.sub("|".join([x for x in [userIdCatcher, discord_emote_regex, url_regex]]), "", comment)
    comment = re.sub(contract_regex, "", comment)
    comment = re.sub(punc_regex, " ", comment)
    comment = comment.replace("\n", " ").replace("'s", "")
    return " " + comment + " "

In [626]:
full_comments = full_comments.assign(cc_comment = lambda x: x.content.apply(clean_comment).apply(sentiment_art))

In [627]:
full_comments = full_comments.assign(pos_sent = lambda x: x.cc_comment.apply(lambda y: y['pos']))\
    .assign(neg_sent = lambda x: x.cc_comment.apply(lambda y: y['neg']))\
        .assign(neu_sent = lambda x: x.cc_comment.apply(lambda y: y['neu']))\
            .assign(comp_sent = lambda x: x.cc_comment.apply(lambda y: y['compound']))

In [628]:
full_comments.drop('cc_comment', axis=1, inplace=True)

In [629]:
company_set = set(x.lower() for x in mentions.symbol.values)

In [630]:
def list_to_symbols(list_obj):
    symbols = []
    for l in list_obj:
        if l in company_set:
            symbols.append(l)
    return symbols

In [631]:
full_comments = full_comments.assign(cc_comment = lambda x: x.content.apply(str.lower))

In [632]:
full_comments = full_comments.assign(symbols = lambda x: x.cc_comment.apply(clean_comment).apply(nltk.tokenize.word_tokenize).apply(lambda z: list_to_symbols(z)).apply(str))

In [633]:
emote_agg = {}
def agg_(comma_companies, agg_dict):
    comma_companies = re.sub(r'[\[\]\'\']', "", comma_companies)
    for e in comma_companies.split(","):
        e = e.strip()
        if e in agg_dict.keys():
            emote_agg[e] += 1
        else:
            emote_agg[e] = 1

In [634]:
full_comments.symbols.apply(lambda x: agg_(x, emote_agg))

0        None
1        None
2        None
3        None
4        None
         ... 
31929    None
31930    None
31931    None
31932    None
31933    None
Name: symbols, Length: 31934, dtype: object

In [635]:
new_mentions = pd.Series(emote_agg).reset_index().iloc[1:, :].sort_values(0, ascending=False).reset_index(drop=True).rename(columns={'index': 'symbol', 0:'counts'})

In [636]:
mentions.merge(new_mentions.assign(symbol=lambda x: x.symbol.apply(str.upper)), 'left', on='symbol').iloc[29:, :]

,symbol,counts_x,counts_y
29,WWR,30,34
30,HYLN,29,40
31,SNDL,29,39
32,JD,29,31
33,SNAP,29,52
34,RKT,28,42
35,ES,27,36
36,JKS,25,46
37,NNDM,25,30
38,GME,24,37


In [303]:
comment_token_dict = {}
# full_comments = full_comments.assign(cc_comment = lambda x: x.content.apply(clean_comment).apply(str.lower))

In [304]:
tokenSeries = wordCounts(full_comments.cc_comment.values).reset_index().rename(columns={'index': 'word', 0:'counts'})

In [305]:
commentsPositiveSeries = wordCounts(full_comments[lambda x: x.comp_sent >= 0.0].cc_comment.values).reset_index().rename(columns={'index': 'word', 0:'counts'})
commentsNegativeSeries = wordCounts(full_comments[lambda x: x.comp_sent <= 0.0].cc_comment.values).reset_index().rename(columns={'index': 'word', 0:'counts'})

In [306]:
commentsNegativeSeries[lambda x: x.word == 'nflx']

,word,counts
791,nflx,17


In [307]:
commentsPositiveSeries[lambda x: x.word == 'nflx']

,word,counts
823,nflx,26
